# **Sales Pirce Study Notebook**

# Objectives

*   Answer business requirement 1: 
    * 1 - The client is interested in discovering how the house attributes correlate with the sale price. Therefore, the client expects data visualisations of the correlated variables against the sale price to show that.
## Inputs

* outputs/datasets/collection/house_prices_records.csv

## Outputs

* generate code and graphs that answers business requirement 1 and can be used to build the Streamlit App

## CRISP-DM
* Data Understanding







---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/milestone-project-heritage-housing-issues/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/milestone-project-heritage-housing-issues'

# Load Data

Load data into pandas data frame

In [4]:
import pandas as pd
df = pd.read_csv("/workspace/milestone-project-heritage-housing-issues/outputs/datasets/collection/house_prices_records.csv")

    
df.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500


---

# Data Exploration

We are looking deeper into the data set to check variable type and distribution, missing levels and what these variables mean in a business context

In [5]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()


/workspace/.pip-modules/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'ipywidgets'

# Corroleaton study

---

Now we will do a test with spearmanns and pearson test to investigate correltaion between the varriables and sales price

In [8]:
corr_spearman = df.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(15)
corr_spearman

OverallQual     0.809829
GrLivArea       0.731310
YearBuilt       0.652682
GarageArea      0.649379
TotalBsmtSF     0.602725
GarageYrBlt     0.593788
1stFlrSF        0.575408
YearRemodAdd    0.571159
OpenPorchSF     0.477561
LotArea         0.456461
MasVnrArea      0.421309
LotFrontage     0.409076
BsmtFinSF1      0.301871
2ndFlrSF        0.298809
WoodDeckSF      0.252176
Name: SalePrice, dtype: float64

In [9]:
corr_pearson = df.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(15)
corr_pearson

OverallQual     0.790982
GrLivArea       0.708624
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
YearBuilt       0.522897
YearRemodAdd    0.507101
GarageYrBlt     0.486362
MasVnrArea      0.477493
BsmtFinSF1      0.386420
LotFrontage     0.351799
2ndFlrSF        0.322335
OpenPorchSF     0.315856
LotArea         0.263843
WoodDeckSF      0.252027
Name: SalePrice, dtype: float64

In both correlation tests, there are strong corrilations in the top 5 for each but peerson shows stong corrilation overall in top 10.

We will consider the top five correlation levels for `df` and will study the associated variables in `df`

## Correlation Study


The functions bellow: 
CalculateCorrAndPPS(): calculates the correlation and PPS for a dataset, and prints the Q1 and Q3 for PPS scores.
DisplayCorrAndPPS(): provides a visualisation of the correlation and PPS of the data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
sns.set_style('whitegrid')


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Display results

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.5, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

From the Spearmann and Perrson corraitaion studdies done, the top 5 most corrilated variables ahve beeen picked.

Spearmann :OverallQual, GrLivArea, YearBuilt, GarageArea, TotalBsmtSF      

Pearson : OverallQual, GrLivArea, GarageArea, TotalBsmtSF, 1stFlrSF        

        

# EDA on selected variables

Now lets explore the most correlated vars when compared to our target var, sale price

In [ ]:
top_n = 5
vars_to_study = set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())
vars_to_study

['KitchenQual_Ex',
 'GrLivArea',
 'MasVnrArea',
 '1stFlrSF',
 'OverallQual',
 'OverallQual',
 'GarageFinish_Unf',
 'TotalBsmtSF',
 'GarageFinish_Fin',
 'BsmtFinType1_GLQ']

In [ ]:
df_eda = df.filter(list(vars_to_study) + ['SalePrice'])
df_eda.head(3)

,1stFlrSF,BsmtFinType1,GarageFinish,GrLivArea,KitchenQual,MasVnrArea,OverallQual,TotalBsmtSF,SalePrice
0,856,GLQ,RFn,1710,Gd,196.0,7,856,208500
1,1262,ALQ,RFn,1262,TA,0.0,6,1262,181500
2,920,GLQ,RFn,1786,Gd,162.0,7,920,223500


First compair each var to sales price individually

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


def plot_scatter(df, col, target_var):
    sns.lmplot(data=df, x=col, y=target_var)
    plt.show()


target_var = 'SalePrice'
for col in vars_to_study:
    plot_scatter(df, col, target_var)
    print("\n\n")

## Multivarable anylsis

In [ ]:
def plot_scatter(df, col, target_var):
    sns.lmplot(data=df, x="GrLivArea", y="SalePrice", ci=None, hue="OverallQual")
    plt.show()
    


target_var = 'SalePrice'
for col in vars_to_study:
    print(f"{col}")
    plot_scatter(df, col, target_var)
    print("\n\n")

## Conclusion

* Larger houses in general, have higher sales prices. \n

* The higher the overall quality of the house the higher the sale price

* Newer houses tent to be more expensive

* The higher the square footage the higher the price


---